In [124]:
import pandas as pd
import xgboost as xgb
from datetime import timedelta
import numpy as np
import joblib

np.set_printoptions(precision=3, suppress=True)

In [125]:
bst = xgb.Booster({'nthread':4}) # init model
bst.load_model("./xgboosting.model")      # load data

In [126]:
test_df = pd.read_csv('../Data/test.csv')

In [127]:
len(tags)

66

In [128]:
cats = joblib.load('./cats.pkl')

In [129]:
genres = joblib.load('./genres.pkl')

In [130]:
tags = joblib.load('./tags.pkl')

In [131]:
def predict_data_prpare(data):
    X = []
    for item in data.iterrows():
        row = item[1]
        tmp = []
        # genres
        row_g = row['genres'].split(',')
        for g in genres:
            if g in row_g:
                tmp.append(1)
            else:
                tmp.append(0)
        # cats
        row_c = row['categories'].split(',')
        for c in cats:
            if c in row_c:
                tmp.append(1)
            else:
                tmp.append(0)
        # tags
        row_t = row['tags'].split(',')
        for t in tags:
            if t in row_t:
                tmp.append(1)
            else:
                tmp.append(0)
        # is_free
        if row['is_free']:
            tmp.append(1)
        else:
            tmp.append(0)
        # price
        tmp.append(row['price'])
        # timegap
        if not np.isnan(row['time_gap']):
            tmp.append(row['time_gap'])
        else:
            tmp.append(0.5)
        # average_review
        if not np.isnan(row['average_review']):
            tmp.append(row['average_review'])
        else:
            tmp.append(0.5)
        
        X.append(tmp)
    
    return X

In [132]:
test_df['average_review'] = test_df['total_positive_reviews'] / (test_df['total_positive_reviews'] + test_df['total_negative_reviews'])

In [133]:
test_df['purchase_date'] = pd.to_datetime(test_df['purchase_date'], infer_datetime_format=True)
test_df['release_date'] = pd.to_datetime(test_df['release_date'], infer_datetime_format=True)

In [134]:
test_df['time_gap'] = test_df['purchase_date'] - test_df['release_date']

In [135]:
test_df['time_gap'] = (test_df['time_gap'] - timedelta(-481)) / (timedelta(4320) - timedelta(-481))

In [136]:
test_df['price'] = test_df['price'] / 15999900.0

In [137]:
# test_df

In [138]:
X = predict_data_prpare(test_df)

In [139]:
test_x = xgb.DMatrix(X)

In [140]:
ans = bst.predict(test_x)

In [141]:
ans

array([ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.002,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.112,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  1.655,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.004,  0.   ,  0.   ,
        0.091,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.   ,  0.009,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , 26.347,
        0.   ,  0.   ,  0.   ,  0.   ,  0.001,  0.007,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.003,  0.   ,  0.017,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
        0.035,  0.   ], dtype=float32)

In [142]:
for i in range(len(ans)):
    ans[i] = round(ans[i], 3)
    if ans[i] > 1:
        ans[i] = 1.0

In [143]:
ans *= 113.8

In [144]:
ans[31]

113.8

In [145]:
ans[63]

113.8

In [146]:
ans

array([  0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.228,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
        12.746,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   , 113.8  ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.455,   0.   ,   0.   ,  10.356,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   1.024,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
       113.8  ,   0.   ,   0.   ,   0.   ,   0.   ,   0.114,   0.797,
         0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.341,   0.   ,
         1.935,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         0.   ,   0.   ,   0.   ,   0.   ,   3.983,   0.   ],
      dtype=float32)

In [147]:
output = pd.DataFrame({'playtime_forever': ans}, columns=['playtime_forever'])

In [148]:
output.to_csv('result5.csv', index_label='id')

In [148]:
t = np.isnan(X)

In [149]:
for i in range(len(t)):
    for k in range(len(t[0])):
        if t[i][k]:
            print(k)

In [192]:
len(X)

90

In [45]:
# all
# array([0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 2.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        1.000000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 1.442250e+03,
#        0.000000e+00, 0.000000e+00, 5.000000e-02, 0.000000e+00,
#        0.000000e+00, 4.060000e+00, 0.000000e+00, 0.000000e+00,
#        2.290000e+01, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 1.000000e-02, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        1.000000e-02, 7.200000e-01, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 3.613406e+04,
#        0.000000e+00, 0.000000e+00, 6.000000e-02, 0.000000e+00,
#        1.400000e-01, 1.700000e-01, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 0.000000e+00, 0.000000e+00, 3.000000e-02,
#        0.000000e+00, 1.120000e+00, 0.000000e+00, 0.000000e+00,
#        0.000000e+00, 1.000000e-02, 0.000000e+00, 1.600000e-01,
#        1.000000e-02, 0.000000e+00, 0.000000e+00, 0.000000e+00,
#        3.180000e+00, 2.000000e-02], dtype=float32)